In [3]:
import pandas as pd
import os
from string import Template

In [41]:
def DIR_CHECK_CREATE(Input_Path):
    if (not os.path.isdir(Input_Path)):
        print ("New Dir Made is" + Input_Path)
        os.makedirs(Input_Path)
    else: 
        print ('Dir Exists')
    return None

def Save_CMD_sh(_str, _out):
    text_file = open(_out, "w")
    text_file.write(_str)
    text_file.close()
    return None

def Save_cmd_sh(_path, _out):
    ## this module generate CMD for any input path of cram
    
    File_Path = _path
    Out_Folder=_out
    Output_shPATH = Out_Folder+ File_Path.split("/")[-1].split(".")[0]+".molecule_table.sh"
    Output_PATH = Out_Folder+ File_Path.split("/")[-1].split(".")[0]+".molecule_table.tsv"
    
    teml = Template("$EXE_PATH --input ${bam_file} --input-bed ${wg_bed_file} \
--reference /ghess/shared/ref/genome.fa --num-threads 8 \
--max-insert 500 --output-csv ${output_file} \n" )

    data={
        "EXE_PATH": "/ghess/groups/algorithms/projects/Study_Fragment_Size/version.beta/ghbip_family_stats",
        "bam_file": File_Path,"output_file": Output_PATH,
        "wg_bed_file": "/home/xili/Working_PATH/Study_Fragment_Size/version.beta/Omni1.0_probes.bed",
    }
    
    
    teml2 = Template("python $PYTHON_PATH --input ${Input_PATH} --output ${Output_Folder} " )
    data2 = {
        "PYTHON_PATH": "/ghess/groups/algorithms/projects/Study_Fragment_Size/version.beta/Generate_FragSize_Distribution.py",
        "Input_PATH": Output_PATH,
        "Output_Folder": Out_Folder
    }
    
    
    text_file = open(Output_shPATH, "w")
    text_file.write(teml.substitute(data))
    text_file.write(teml2.substitute(data2))
    text_file.close()
    return Output_shPATH, File_Path.split("/")[-1].split(".")[0]

def Run_qsub(_path, _name):
    data = {
        "OUTPATH": _path,
        "NAME": _name
    }
    teml = Template("qsub -V -q dev.q -j y -b y -cwd -o ${OUTPATH}.log -N ${NAME} -l mem_free=8G,h_vmem=8G -pe parallel 8 sh ${OUTPATH}" )
    
    os.system(teml.substitute(data))
    return None

In [42]:
PATH = '/ghess/groups/algorithms/projects/Study_Fragment_Size/Sample/List_of_samples_4TF_training.210708.tsv'
PATH = "/ghess/groups/algorithms/projects/Study_Fragment_Size/Sample/List_of_samples_4TF_testing.210708.tsv"
df_sum = pd.read_csv(PATH, sep='\t')

i=0
Out_Path="/ghess/groups/algorithms/projects/Study_Fragment_Size/Sample/Testing_Sample/molecule_counts/"
DIR_CHECK_CREATE(Out_Path)
for path in df_sum.bam_file:
    i+=1
    if(os.path.isfile(path)):
        #print("File is :", path)
        path_sh, name = Save_cmd_sh(path, Out_Path)
        Run_qsub(path_sh, name)
    #if(i==1):break

Dir Exists


In [19]:
df_tem = pd.read_csv("Omni1.0_probes.bed", sep='\t', skiprows=1, header=None)

df_tem[5][0]

'GENE=FAAP20;GC_CONTENT=0.617;STRAND=+;REPLICATION=1;PROBESET=backbone'